In [2]:
import grid2op
import numpy as np
import torch
from grid2op.PlotGrid import PlotMatplot
from grid2op.Converter import IdToAct

In [3]:
env = grid2op.make("l2rpn_neurips_2020_track2_small")

In [5]:
action_space = IdToAct(env.action_space)
action_space.init_converter(all_actions='../data/l2rpn_neurips_2020_track2_small_action_space.npy')

In [6]:
len(action_space.all_actions)

73108

In [10]:
act = action_space.all_actions[1000]
impacts = act.impact_on_objects()

In [11]:
impacts

{'has_impact': True,
 'injection': {'changed': False, 'count': 0, 'impacted': []},
 'force_line': {'changed': False,
  'reconnections': {'count': 0, 'powerlines': []},
  'disconnections': {'count': 0, 'powerlines': []}},
 'switch_line': {'changed': False, 'count': 0, 'powerlines': []},
 'topology': {'changed': True,
  'bus_switch': [],
  'assigned_bus': [{'bus': 2,
    'object_type': 'line (extremity)',
    'object_id': 2,
    'substation': 11},
   {'bus': 1,
    'object_type': 'line (extremity)',
    'object_id': 13,
    'substation': 11},
   {'bus': 2,
    'object_type': 'line (extremity)',
    'object_id': 24,
    'substation': 11},
   {'bus': 2,
    'object_type': 'line (extremity)',
    'object_id': 35,
    'substation': 11},
   {'bus': 2,
    'object_type': 'line (origin)',
    'object_id': 57,
    'substation': 11},
   {'bus': 1,
    'object_type': 'line (origin)',
    'object_id': 80,
    'substation': 11},
   {'bus': 1,
    'object_type': 'line (origin)',
    'object_id': 83,


In [13]:
action_line_matrix = np.zeros((len(action_space.all_actions), env.n_line))

In [15]:
for i, a in enumerate(action_space.all_actions):
    impacts = a.impact_on_objects()
    if impacts['force_line']['changed']:
        reconn_lines = impacts['force_line']['reconnections']['powerlines']
        if len(reconn_lines) > 0:
            action_line_matrix[i, reconn_lines] = 1
    if impacts['topology']['changed']:
        set_bus_lines = [obj['object_id'] for obj in impacts['topology']['assigned_bus'] if obj['object_type'].startswith('line')]
        if len(set_bus_lines) > 0:
            action_line_matrix[i, set_bus_lines] = 1

In [16]:
np.save('../data/l2rpn_neurips_2020_track2_small_action_x_line.numpy', action_line_matrix)

In [18]:
np.where(action_line_matrix[1000] == 1)

(array([ 2, 13, 24, 35, 57, 80, 83]),)

In [19]:
action_line_matrix.shape

(73108, 186)